In [30]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading

In [31]:
jumlah_index = 200
threads_link = []
links = []
results = []
threads = []
keywords='anies'

In [32]:
def scrape_links(page_number,keywords):
    url = f"https://www.cnbcindonesia.com/search?query={keywords}&p={page_number}&kanal=&tipe=&date="
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article')
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [33]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p,keywords)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 12 links from page 26
Scraped 12 links from page 31
Scraped 12 links from page 17
Scraped 12 links from page 29
Scraped 12 links from page 25
Scraped 12 links from page 23
Scraped 12 links from page 20
Scraped 12 links from page 35
Scraped 12 links from page 15
Scraped 12 links from page 37
Scraped 12 links from page 36
Scraped 12 links from page 30
Scraped 12 links from page 24
Scraped 12 links from page 45
Scraped 12 links from page 28
Scraped 12 links from page 4
Scraped 12 links from page 43
Scraped 12 links from page 21
Scraped 12 links from page 18
Scraped 12 links from page 52
Scraped 12 links from page 57
Scraped 12 links from page 64
Scraped 12 links from page 86
Scraped 12 links from page 94
Scraped 12 links from page 75
Scraped 12 links from page 59
Scraped 12 links from page 90
Scraped 12 links from page 83
Scraped 12 links from page 67
Scraped 12 links from page 88
Scraped 12 links from page 77
Scraped 12 links from page 53
Scraped 12 links from page 78
Scraped 12 

In [34]:
results = []
def scrape_url(url,keywords):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            
            # Judul Berita
            title_elem = soup.find('h1')
            if title_elem:
                title_text = title_elem.text.strip()
            else:
                title_text = "Title not found"
            # Author berita
            author_elem = soup.find('div', {"class": "author"})
            if author_elem:
                author_text = author_elem.get_text()
                author_text = author_text.split('-')[1].strip()
                author_text = author_text.split(',')[0].strip()
            else:
                author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('div', {"class": "date"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            category_elements = soup.find('ul', {"class": "breadcrumb"})
            if category_elements:
                category_text = category_elements.find_all('li')
                category_text= category_text[2].get_text()
            else:
                category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "detail_text"})
            
            if body_elem:
                content_elem = body_elem.find_all('p')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
              content_text ="Content not found"

            results.append({'title': title_text,
                            'keywords': keywords,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [35]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,keywords))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [36]:
df = pd.DataFrame(results)
print('hasil scrapping',len(results))
df.head(10)

hasil scrapping 2400


,title,keywords,author,category,date,content,link
0,Capres Ini Punya Elektabilitas Tinggi dan Raji...,anies,Robertus Andrianto,Berita MyMoney,14 August 2023 14:05,"Jakarta, CNBC Indonesia - Prabowo Subianto di...",https://www.cnbcindonesia.com/mymoney/20230814...
1,"Media Asing Soroti Pilpres RI, Heboh Capres In...",anies,Tommy Patrio Sorongan,Berita,14 August 2023 08:00,"Jakarta, CNBC Indonesia - Pemilihan Presiden (...",https://www.cnbcindonesia.com/news/20230814072...
2,"Media Asing Sorot Pilpres RI, Sebut Capres Dik...",anies,sef,Berita,03 August 2023 10:04,"Jakarta, CNBC Indonesia - Media asing menyorot...",https://www.cnbcindonesia.com/news/20230803100...
3,"Bukan Anies-Ganjar-Prabowo, Asing Sorot Pilpre...",anies,Novina Putri Bestari & redaksi,Berita,12 August 2023 07:30,"Jakarta, CNBC Indonesia - Pemilihan presiden (...",https://www.cnbcindonesia.com/news/20230812062...
4,"Asing Sorot Pilpres RI, Bukan Anies-Prabowo-Ga...",anies,Thea Fathanah Arbar,Berita,11 August 2023 12:00,"Jakarta, CNBC Indonesia - Media asing kembali ...",https://www.cnbcindonesia.com/news/20230811102...
5,Cawapres Anies Diajak Gabung Jadi Timses Ganja...,anies,Wiji Nur Hayat,Berita,03 August 2023 06:15,"Jakarta, CNBC Indonesia - Salah satu kandidat ...",https://www.cnbcindonesia.com/news/20230802195...
6,Top! Ini Capres Pilihan 6 Juta Peternak Ayam S...,anies,linda hasibuan,Berita,13 August 2023 13:00,"Jakarta, CNBC Indonesia - Menjelang Pemilu Pre...",https://www.cnbcindonesia.com/news/20230813123...
7,JIS Dipakai Piala Dunia U-17 Hingga Media Asin...,anies,cnbc indonesia,Video News,02 August 2023 20:30,"Jakarta, CNBC Indonesia - Mantan Gubernur DKI ...",https://www.cnbcindonesia.com/news/20230802182...
8,"Peta Koalisi Terkini Partai Pendukung Prabowo,...",anies,Tim Redaksi,Berita,14 August 2023 13:25,"Jakarta, CNBC Indonesia - Partai Golongan Kary...",https://www.cnbcindonesia.com/news/20230814111...
9,Kandidat Cawapres Anies Diajak Gabung Timses G...,anies,Wiji Nur Hayat,Berita,02 August 2023 12:16,"Jakarta, CNBC Indonesia - Salah satu kandidat ...",https://www.cnbcindonesia.com/news/20230802121...


Save File

In [37]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
excel_file_name = f'./excel/cnbc_{keywords}_{current_datetime}.xlsx'
df.to_excel(excel_file_name, index=False)

print(f'Data has been saved to {excel_file_name}')

Data has been saved to ./excel/cnbc_anies_2023-09-27_07-06-53.xlsx


In [38]:
# Tempat Searching dan cek web
# url = "https://www.cnbcindonesia.com/news/20230924184706-4-475109/kota-hantu-china-bikin-pening-ada-72-juta-rumah-tak-laku"  
# response = requests.get(url)
# soup = BeautifulSoup(response.text, "html.parser")
# # author_elem = soup.find('ul', {"class": "breadcrumb"})
# author_elem = soup.find('ul', {"class": "breadcrumb"})

# author_text = author_elem.find_all('li')
# # author_text = author_text.split('-')[1].strip()
# # author_text = author_text.split(',')[0].strip()
# print(author_text[2].get_text())


